In [119]:
import sys
sys.path.append("..")
from lib.util import add_race_eth,  add_income_group
import pandas as pd
pd.options.display.max_columns = 50
pd.options.display.max_rows = 1550
import numpy as np
from ambry import library
l = library()
p = l.get('ffiec.gov-hmda-2010e-lar-2012-ca').partition
df = p.select("SELECT * FROM lar WHERE county_code = 73").pandas

In [120]:
add_race_eth(df)
dfs = df[['id','census_tract_number', 'applicant_ethnicity','applicant_race_1', 'race_eth']]

# All Lenders Combined
- **Missing: Minority Status, Tract/BNA**

In [121]:
###
# Action type == ALL
###
totalcount = df.loan_amount.count()
totalamount = df.loan_amount.sum()

### By Loan Purpose and Type

df['loan_purtype'] = '_'
df['loan_purtype'][(df['loan_purpose'] == 1) & (df['loan_type'] == 1)] = 'Purchase - Conventional'
df['loan_purtype'][(df['loan_purpose'] == 1) & (df['loan_type'] > 1)] = 'Purchase - Goverment'
df['loan_purtype'][df['loan_purpose'] == 2] = 'Home Improvement'
df['loan_purtype'][df['loan_purpose'] == 3] = 'Refinancing'

loanpt = df.groupby(['loan_purtype'])
loanpt = loanpt['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})
loanpt['%count'] = np.round(100*loanpt['count']/totalcount,1)
loanpt['%amount'] = np.round(100*loanpt['amount']/totalamount,1)
loanpt=loanpt[['count','%count','amount','%amount']]

### By Applicant Race/Ethnicity

race_eth  = df.groupby(['race_eth'])
race_eth  = race_eth ['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

race_eth ['%count'] = np.round(100*race_eth['count']/totalcount,1)
race_eth ['%amount'] = np.round(100*race_eth['amount']/totalamount,1)
race_eth = race_eth [['count','%count','amount','%amount']]

###By Applicant Income

df['applicant_income_percent_median'] = 100*df.applicant_income/df.applicant_income.median()
df['applicant_income_cat'] = 'Income Not Available'
df['applicant_income_cat'][df['applicant_income_percent_median'] < 50] = 'Low'
df['applicant_income_cat'][(df['applicant_income_percent_median'] >= 50) & (df['applicant_income_percent_median'] < 80)] = 'Moderate'
df['applicant_income_cat'][(df['applicant_income_percent_median'] >= 80) & (df['applicant_income_percent_median'] < 120)] = 'Middle'
df['applicant_income_cat'][df['applicant_income_percent_median'] >= 120] = 'Upper'

income  = df.groupby(['applicant_income_cat'])
income   = income['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

income ['%count'] = np.round(100*income['count']/totalcount,1)
income ['%amount'] = np.round(100*income['amount']/totalamount,1)
income = income [['count','%count','amount','%amount']]

###By Applicant Sex

sex  = df.groupby(['applicant_sex'])
sex   = sex['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

sex ['%count'] = np.round(100*sex['count']/totalcount,1)
sex ['%amount'] = np.round(100*sex['amount']/totalamount,1)
sex = sex [['count','%count','amount','%amount']]

### Smashing Tables Together

frames = [loanpt, race_eth,income,sex]
totalapp = pd.concat(frames, keys=['Loan Purpose and Type', 
                                   'Applicant Race/Eth', 
                                   'Applicant Income',
                                   'Applicant Sex'])
totalapp
pieces = [totalapp.ix[:, ["count","%count","amount","%amount",]]]
totalapp = pd.concat(pieces, axis=1, keys=['Total Applications (1)'])
totalapp

Total Applications (1)          \
                                                                count  %count   
Loan Purpose and Type Home Improvement                           4781     2.0   
                      Purchase - Conventional                   32857    13.5   
                      Purchase - Goverment                      22240     9.2   
                      Refinancing                              182905    75.3   
Applicant Race/Eth    aian                                        767     0.3   
                      as                                        23920     9.9   
                      b                                          4053     1.7   
                      h                                         24012     9.9   
                      pi                                         1908     0.8   
                      w                                        142000    58.5   
Applicant Income      Income Not Available                      28473    11.7   
                      Low                                       26217    10.8   
                      Middle                                    57469    23.7   
                      Moderate                                  49857    20.5   
                      Upper                                     80767    33.3   
Applicant Sex         1                                        159485    65.7   
                      2                                         50394    20.8   
                      3                                         17527     7.2   
                      4                                         15377     6.3   

                                                                  
                                                 amount  %amount  
Loan Purpose and Type Home Improvement          1038687      1.3  
                      Purchase - Conventional  12419787     15.1  
                      Purchase - Goverment      7551607      9.2  
                      Refinancing              61055963     74.4  
Applicant Race/Eth    aian                       231319      0.3  
                      as                        7808190      9.5  
                      b                         1139822      1.4  
                      h                         6385159      7.8  
                      pi                         570463      0.7  
                      w                        48384565     59.0  
Applicant Income      Income Not Available     11876933     14.5  
                      Low                       4777981      5.8  
                      Middle                   17701587     21.6  
                      Moderate                 12375557     15.1  
                      Upper                    35333986     43.1  
Applicant Sex         1                        54930806     66.9  
                      2                        14172171     17.3  
                      3                         5939329      7.2  
                      4                         7023738      8.6

In [122]:
#######
# Action Type =1
#######
df1 = df[df.action_type==1]
totalcount = df1.loan_amount.count()
totalamount = df1.loan_amount.sum()

### By Loan Purpose and Type
df1['loan_purtype'] = '_'
df1['loan_purtype'][(df1['loan_purpose'] == 1) & (df1['loan_type'] == 1)] = 'Purchase - Conventional'
df1['loan_purtype'][(df1['loan_purpose'] == 1) & (df1['loan_type'] > 1)] = 'Purchase - Goverment'
df1['loan_purtype'][df1['loan_purpose'] == 2] = 'Home Improvement'
df1['loan_purtype'][df1['loan_purpose'] == 3] = 'Refinancing'

loanpt = df1.groupby(['loan_purtype'])
loanpt = loanpt['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})
loanpt['%count'] = np.round(100*loanpt['count']/totalcount,1)
loanpt['%amount'] = np.round(100*loanpt['amount']/totalamount,1)
loanpt=loanpt[['count','%count','amount','%amount']]

### By Applicant Race/Ethnicity

race_eth  = df1.groupby(['race_eth'])
race_eth  = race_eth ['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

race_eth ['%count'] = np.round(100*race_eth['count']/totalcount,1)
race_eth ['%amount'] = np.round(100*race_eth['amount']/totalamount,1)
race_eth = race_eth [['count','%count','amount','%amount']]


###By Applicant Income

df1['applicant_income_percent_median'] = 100*df1.applicant_income/df1.applicant_income.median()
df1['applicant_income_cat'] = 'Income Not Available'
df1['applicant_income_cat'][df1['applicant_income_percent_median'] < 50] = 'Low'
df1['applicant_income_cat'][(df1['applicant_income_percent_median'] >= 50) & (df1['applicant_income_percent_median'] < 80)] = 'Moderate'
df1['applicant_income_cat'][(df1['applicant_income_percent_median'] >= 80) & (df1['applicant_income_percent_median'] < 120)] = 'Middle'
df1['applicant_income_cat'][df1['applicant_income_percent_median'] >= 120] = 'Upper'

income  = df1.groupby(['applicant_income_cat'])
income   = income['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

income ['%count'] = np.round(100*income['count']/totalcount,1)
income ['%amount'] = np.round(100*income['amount']/totalamount,1)
income = income [['count','%count','amount','%amount']]

###By Applicant Sex

sex  = df1.groupby(['applicant_sex'])
sex   = sex['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

sex ['%count'] = np.round(100*sex['count']/totalcount,1)
sex ['%amount'] = np.round(100*sex['amount']/totalamount,1)
sex = sex [['count','%count','amount','%amount']]

### Smashing Tables Together

frames = [loanpt, race_eth,income,sex]
totalapp1 = pd.concat(frames, keys=['Loan Purpose and Type', 
                                   'Applicant Race/Eth', 
                                   'Applicant Income',
                                   'Applicant Sex'])
pieces = [totalapp1.ix[:, ["count","%count","amount","%amount",]]]
totalapp1 = pd.concat(pieces, axis=1, keys=['Loan originated'])
totalapp1

/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:9: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead
/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:37: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead
/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:38: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead


Loan originated          \
                                                         count  %count   
Loan Purpose and Type Home Improvement                    2315     1.7   
                      Purchase - Conventional            18452    13.7   
                      Purchase - Goverment               10545     7.8   
                      Refinancing                       103541    76.8   
Applicant Race/Eth    aian                                 417     0.3   
                      as                                 14274    10.6   
                      b                                   2219     1.6   
                      h                                  12799     9.5   
                      pi                                  1027     0.8   
                      w                                  86821    64.4   
Applicant Income      Income Not Available               12748     9.5   
                      Low                                14379    10.7   
                      Middle                             33853    25.1   
                      Moderate                           28098    20.8   
                      Upper                              45775    33.9   
Applicant Sex         1                                  95760    71.0   
                      2                                  29277    21.7   
                      3                                   8844     6.6   
                      4                                    972     0.7   

                                                                  
                                                 amount  %amount  
Loan Purpose and Type Home Improvement           624517      1.3  
                      Purchase - Conventional   7434158     16.0  
                      Purchase - Goverment      3629798      7.8  
                      Refinancing              34805947     74.9  
Applicant Race/Eth    aian                       127627      0.3  
                      as                        4702467     10.1  
                      b                          630383      1.4  
                      h                         3417863      7.4  
                      pi                         309668      0.7  
                      w                        29334688     63.1  
Applicant Income      Income Not Available      6322209     13.6  
                      Low                       2606371      5.6  
                      Middle                   10424061     22.4  
                      Moderate                  6996014     15.0  
                      Upper                    20145765     43.3  
Applicant Sex         1                        32921809     70.8  
                      2                         8218736     17.7  
                      3                         3100921      6.7  
                      4                         2252954      4.8

In [123]:
#######
# Action Type =2 
#######
df2 = df[df.action_type==2]
totalcount = df2.loan_amount.count()
totalamount = df2.loan_amount.sum()

### By Loan Purpose and Type

df2['loan_purtype'] = '_'
df2['loan_purtype'][(df2['loan_purpose'] == 1) & (df2['loan_type'] == 1)] = 'Purchase - Conventional'
df2['loan_purtype'][(df2['loan_purpose'] == 1) & (df2['loan_type'] > 1)] = 'Purchase - Goverment'
df2['loan_purtype'][df2['loan_purpose'] == 2] = 'Home Improvement'
df2['loan_purtype'][df2['loan_purpose'] == 3] = 'Refinancing'

loanpt = df2.groupby(['loan_purtype'])
loanpt = loanpt['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})
loanpt['%count'] = np.round(100*loanpt['count']/totalcount,1)
loanpt['%amount'] = np.round(100*loanpt['amount']/totalamount,1)
loanpt=loanpt[['count','%count','amount','%amount']]

### By Applicant Race/Ethnicity

race_eth  = df2.groupby(['race_eth'])
race_eth  = race_eth ['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

race_eth ['%count'] = np.round(100*race_eth['count']/totalcount,1)
race_eth ['%amount'] = np.round(100*race_eth['amount']/totalamount,1)
race_eth = race_eth [['count','%count','amount','%amount']]

###By Applicant Income

df2['applicant_income_percent_median'] = 100*df2.applicant_income/df2.applicant_income.median()
df2['applicant_income_cat'] = 'Income Not Available'
df2['applicant_income_cat'][df2['applicant_income_percent_median'] < 50] = 'Low'
df2['applicant_income_cat'][(df2['applicant_income_percent_median'] >= 50) & (df2['applicant_income_percent_median'] < 80)] = 'Moderate'
df2['applicant_income_cat'][(df2['applicant_income_percent_median'] >= 80) & (df2['applicant_income_percent_median'] < 120)] = 'Middle'
df2['applicant_income_cat'][df2['applicant_income_percent_median'] >= 120] = 'Upper'

income  = df2.groupby(['applicant_income_cat'])
income   = income['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

income ['%count'] = np.round(100*income['count']/totalcount,1)
income ['%amount'] = np.round(100*income['amount']/totalamount,1)
income = income [['count','%count','amount','%amount']]

###By Applicant Sex

sex  = df2.groupby(['applicant_sex'])
sex   = sex['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

sex ['%count'] = np.round(100*sex['count']/totalcount,1)
sex ['%amount'] = np.round(100*sex['amount']/totalamount,1)
sex = sex [['count','%count','amount','%amount']]

### Smashing Tables Together

frames = [loanpt, race_eth,income,sex]
totalapp2 = pd.concat(frames, keys=['Loan Purpose and Type', 
                                   'Applicant Race/Eth', 
                                   'Applicant Income',
                                   'Applicant Sex'])
pieces = [totalapp2.ix[:, ["count","%count","amount","%amount",]]]
totalapp2 = pd.concat(pieces, axis=1, keys=['App approved but not accepted'])
totalapp2

/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:10: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead
/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:37: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead
/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:38: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead


App approved but not accepted  \
                                                                       count   
Loan Purpose and Type Home Improvement                                   221   
                      Purchase - Conventional                           1918   
                      Purchase - Goverment                               747   
                      Refinancing                                       6872   
Applicant Race/Eth    aian                                                40   
                      as                                                1185   
                      b                                                  178   
                      h                                                 1114   
                      pi                                                  71   
                      w                                                 5841   
Applicant Income      Income Not Available                               699   
                      Low                                               1157   
                      Middle                                            2403   
                      Moderate                                          2009   
                      Upper                                             3490   
Applicant Sex         1                                                 6732   
                      2                                                 2229   
                      3                                                  765   
                      4                                                   32   

                                                                         
                                               %count   amount  %amount  
Loan Purpose and Type Home Improvement            2.3    31193      1.0  
                      Purchase - Conventional    19.7   656007     20.6  
                      Purchase - Goverment        7.7   245548      7.7  
                      Refinancing                70.4  2255775     70.7  
Applicant Race/Eth    aian                        0.4    12336      0.4  
                      as                         12.1   380775     11.9  
                      b                           1.8    46575      1.5  
                      h                          11.4   286285      9.0  
                      pi                          0.7    21871      0.7  
                      w                          59.9  1955672     61.3  
Applicant Income      Income Not Available        7.2   264188      8.3  
                      Low                        11.9   178175      5.6  
                      Middle                     24.6   717774     22.5  
                      Moderate                   20.6   465712     14.6  
                      Upper                      35.8  1562674     49.0  
Applicant Sex         1                          69.0  2292781     71.9  
                      2                          22.8   594259     18.6  
                      3                           7.8   262191      8.2  
                      4                           0.3    39292      1.2

In [124]:
#######
# Action Type =3 
#######
df3 = df[df.action_type==3]
totalcount = df3.loan_amount.count()
totalamount = df3.loan_amount.sum()

### By Loan Purpose and Type

df3['loan_purtype'] = '_'
df3['loan_purtype'][(df3['loan_purpose'] == 1) & (df3['loan_type'] == 1)] = 'Purchase - Conventional'
df3['loan_purtype'][(df3['loan_purpose'] == 1) & (df3['loan_type'] > 1)] = 'Purchase - Goverment'
df3['loan_purtype'][df3['loan_purpose'] == 2] = 'Home Improvement'
df3['loan_purtype'][df3['loan_purpose'] == 3] = 'Refinancing'

loanpt = df3.groupby(['loan_purtype'])
loanpt = loanpt['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})
loanpt['%count'] = np.round(100*loanpt['count']/totalcount,1)
loanpt['%amount'] = np.round(100*loanpt['amount']/totalamount,1)
loanpt=loanpt[['count','%count','amount','%amount']]

### By Applicant Race/Ethnicity

race_eth  = df3.groupby(['race_eth'])
race_eth  = race_eth ['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

race_eth ['%count'] = np.round(100*race_eth['count']/totalcount,1)
race_eth ['%amount'] = np.round(100*race_eth['amount']/totalamount,1)
race_eth = race_eth [['count','%count','amount','%amount']]

###By Applicant Income

df3['applicant_income_percent_median'] = 100*df3.applicant_income/df3.applicant_income.median()
df3['applicant_income_cat'] = 'Income Not Available'
df3['applicant_income_cat'][df3['applicant_income_percent_median'] < 50] = 'Low'
df3['applicant_income_cat'][(df3['applicant_income_percent_median'] >= 50) & (df3['applicant_income_percent_median'] < 80)] = 'Moderate'
df3['applicant_income_cat'][(df3['applicant_income_percent_median'] >= 80) & (df3['applicant_income_percent_median'] < 120)] = 'Middle'
df3['applicant_income_cat'][df3['applicant_income_percent_median'] >= 120] = 'Upper'

income  = df3.groupby(['applicant_income_cat'])
income   = income['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

income ['%count'] = np.round(100*income['count']/totalcount,1)
income ['%amount'] = np.round(100*income['amount']/totalamount,1)
income = income [['count','%count','amount','%amount']]

###By Applicant Sex

sex  = df3.groupby(['applicant_sex'])
sex   = sex['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

sex ['%count'] = np.round(100*sex['count']/totalcount,1)
sex ['%amount'] = np.round(100*sex['amount']/totalamount,1)
sex = sex [['count','%count','amount','%amount']]

### Smashing Tables Together

frames = [loanpt, race_eth,income,sex]
totalapp3 = pd.concat(frames, keys=['Loan Purpose and Type', 
                                   'Applicant Race/Eth', 
                                   'Applicant Income',
                                   'Applicant Sex'])
pieces = [totalapp3.ix[:, ["count","%count","amount","%amount",]]]
totalapp3 = pd.concat(pieces, axis=1, keys=['App denied by financial institution'])
totalapp3

/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:10: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead
/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:37: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead
/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:38: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead


App denied by financial institution  \
                                                                             count   
Loan Purpose and Type Home Improvement                                        1243   
                      Purchase - Conventional                                 3046   
                      Purchase - Goverment                                    1941   
                      Refinancing                                            23336   
Applicant Race/Eth    aian                                                     132   
                      as                                                      3167   
                      b                                                        714   
                      h                                                       3958   
                      pi                                                       314   
                      w                                                      16516   
Applicant Income      Income Not Available                                    1884   
                      Low                                                     4267   
                      Middle                                                  6555   
                      Moderate                                                5824   
                      Upper                                                  11036   
Applicant Sex         1                                                      19102   
                      2                                                       7148   
                      3                                                       3219   
                      4                                                         97   

                                                                         
                                               %count   amount  %amount  
Loan Purpose and Type Home Improvement            4.2   123326      1.3  
                      Purchase - Conventional    10.3  1057959     11.0  
                      Purchase - Goverment        6.6   638008      6.6  
                      Refinancing                78.9  7781823     81.1  
Applicant Race/Eth    aian                        0.4    35838      0.4  
                      as                         10.7   969824     10.1  
                      b                           2.4   181171      1.9  
                      h                          13.4   989168     10.3  
                      pi                          1.1    86320      0.9  
                      w                          55.9  5711810     59.5  
Applicant Income      Income Not Available        6.4   739228      7.7  
                      Low                        14.4   785670      8.2  
                      Middle                     22.2  1884538     19.6  
                      Moderate                   19.7  1343841     14.0  
                      Upper                      37.3  4847839     50.5  
Applicant Sex         1                          64.6  6553051     68.3  
                      2                          24.2  1934387     20.1  
                      3                          10.9   965108     10.1  
                      4                           0.3   148570      1.5

In [125]:
#######
# Action Type =4
#######
df4 = df[df.action_type==4]
totalcount = df4.loan_amount.count()
totalamount = df4.loan_amount.sum()

### By Loan Purpose and Type

df4['loan_purtype'] = '_'
df4['loan_purtype'][(df4['loan_purpose'] == 1) & (df4['loan_type'] == 1)] = 'Purchase - Conventional'
df4['loan_purtype'][(df4['loan_purpose'] == 1) & (df4['loan_type'] > 1)] = 'Purchase - Goverment'
df4['loan_purtype'][df4['loan_purpose'] == 2] = 'Home Improvement'
df4['loan_purtype'][df4['loan_purpose'] == 3] = 'Refinancing'

loanpt = df4.groupby(['loan_purtype'])
loanpt = loanpt['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})
loanpt['%count'] = np.round(100*loanpt['count']/totalcount,1)
loanpt['%amount'] = np.round(100*loanpt['amount']/totalamount,1)
loanpt=loanpt[['count','%count','amount','%amount']]

### By Applicant Race/Ethnicity

race_eth  = df4.groupby(['race_eth'])
race_eth  = race_eth ['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

race_eth ['%count'] = np.round(100*race_eth['count']/totalcount,1)
race_eth ['%amount'] = np.round(100*race_eth['amount']/totalamount,1)
race_eth = race_eth [['count','%count','amount','%amount']]

###By Applicant Income

df4['applicant_income_percent_median'] = 100*df4.applicant_income/df4.applicant_income.median()
df4['applicant_income_cat'] = 'Income Not Available'
df4['applicant_income_cat'][df4['applicant_income_percent_median'] < 50] = 'Low'
df4['applicant_income_cat'][(df4['applicant_income_percent_median'] >= 50) & (df4['applicant_income_percent_median'] < 80)] = 'Moderate'
df4['applicant_income_cat'][(df4['applicant_income_percent_median'] >= 80) & (df4['applicant_income_percent_median'] < 120)] = 'Middle'
df4['applicant_income_cat'][df4['applicant_income_percent_median'] >= 120] = 'Upper'

income  = df4.groupby(['applicant_income_cat'])
income   = income['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

income ['%count'] = np.round(100*income['count']/totalcount,1)
income ['%amount'] = np.round(100*income['amount']/totalamount,1)
income = income [['count','%count','amount','%amount']]

###By Applicant Sex

sex  = df4.groupby(['applicant_sex'])
sex   = sex['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

sex ['%count'] = np.round(100*sex['count']/totalcount,1)
sex ['%amount'] = np.round(100*sex['amount']/totalamount,1)
sex = sex [['count','%count','amount','%amount']]

### Smashing Tables Together

frames = [loanpt, race_eth,income,sex]
totalapp4 = pd.concat(frames, keys=['Loan Purpose and Type', 
                                   'Applicant Race/Eth', 
                                   'Applicant Income',
                                   'Applicant Sex'])
pieces = [totalapp4.ix[:, ["count","%count","amount","%amount",]]]
totalapp4 = pd.concat(pieces, axis=1, keys=['App withdrawn by applicant'])
totalapp4

/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:10: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead
/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:37: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead
/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:38: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead


App withdrawn by applicant  \
                                                                    count   
Loan Purpose and Type Home Improvement                                337   
                      Purchase - Conventional                        2855   
                      Purchase - Goverment                           1641   
                      Refinancing                                   15794   
Applicant Race/Eth    aian                                             66   
                      as                                             1905   
                      b                                               363   
                      h                                              2277   
                      pi                                              182   
                      w                                             11777   
Applicant Income      Income Not Available                           1644   
                      Low                                            2398   
                      Middle                                         5159   
                      Moderate                                       4337   
                      Upper                                          7089   
Applicant Sex         1                                             13420   
                      2                                              4509   
                      3                                              2648   
                      4                                                50   

                                                                         
                                               %count   amount  %amount  
Loan Purpose and Type Home Improvement            1.6    72603      1.0  
                      Purchase - Conventional    13.8  1117411     15.7  
                      Purchase - Goverment        8.0   546526      7.7  
                      Refinancing                76.6  5376550     75.6  
Applicant Race/Eth    aian                        0.3    20477      0.3  
                      as                          9.2   621148      8.7  
                      b                           1.8   103774      1.5  
                      h                          11.0   607909      8.5  
                      pi                          0.9    52066      0.7  
                      w                          57.1  4120772     57.9  
Applicant Income      Income Not Available        8.0   751373     10.6  
                      Low                        11.6   452626      6.4  
                      Middle                     25.0  1607737     22.6  
                      Moderate                   21.0  1089146     15.3  
                      Upper                      34.4  3212208     45.2  
Applicant Sex         1                          65.1  4695145     66.0  
                      2                          21.9  1289115     18.1  
                      3                          12.8   902962     12.7  
                      4                           0.2   225868      3.2

In [126]:
#######
# Action Type =5
#######

df5 = df[df.action_type==5]
totalcount = df5.loan_amount.count()
totalamount = df5.loan_amount.sum()

### By Loan Purpose and Type

df5['loan_purtype'] = '_'
df5['loan_purtype'][(df5['loan_purpose'] == 1) & (df5['loan_type'] == 1)] = 'Purchase - Conventional'
df5['loan_purtype'][(df5['loan_purpose'] == 1) & (df5['loan_type'] > 1)] = 'Purchase - Goverment'
df5['loan_purtype'][df5['loan_purpose'] == 2] = 'Home Improvement'
df5['loan_purtype'][df5['loan_purpose'] == 3] = 'Refinancing'

loanpt = df5.groupby(['loan_purtype'])
loanpt = loanpt['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})
loanpt['%count'] = np.round(100*loanpt['count']/totalcount,1)
loanpt['%amount'] = np.round(100*loanpt['amount']/totalamount,1)
loanpt=loanpt[['count','%count','amount','%amount']]

### By Applicant Race/Ethnicity

race_eth  = df5.groupby(['race_eth'])
race_eth  = race_eth ['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

race_eth ['%count'] = np.round(100*race_eth['count']/totalcount,1)
race_eth ['%amount'] = np.round(100*race_eth['amount']/totalamount,1)
race_eth = race_eth [['count','%count','amount','%amount']]

###By Applicant Income

df5['applicant_income_percent_median'] = 100*df5.applicant_income/df5.applicant_income.median()
df5['applicant_income_cat'] = 'Income Not Available'
df5['applicant_income_cat'][df5['applicant_income_percent_median'] < 50] = 'Low'
df5['applicant_income_cat'][(df5['applicant_income_percent_median'] >= 50) & (df5['applicant_income_percent_median'] < 80)] = 'Moderate'
df5['applicant_income_cat'][(df5['applicant_income_percent_median'] >= 80) & (df5['applicant_income_percent_median'] < 120)] = 'Middle'
df5['applicant_income_cat'][df5['applicant_income_percent_median'] >= 120] = 'Upper'

income  = df5.groupby(['applicant_income_cat'])
income   = income['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

income ['%count'] = np.round(100*income['count']/totalcount,1)
income ['%amount'] = np.round(100*income['amount']/totalamount,1)
income = income [['count','%count','amount','%amount']]

###By Applicant Sex

sex  = df5.groupby(['applicant_sex'])
sex   = sex['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

sex ['%count'] = np.round(100*sex['count']/totalcount,1)
sex ['%amount'] = np.round(100*sex['amount']/totalamount,1)
sex = sex [['count','%count','amount','%amount']]

### Smashing Tables Together

frames = [loanpt, race_eth,income,sex]
totalapp5 = pd.concat(frames, keys=['Loan Purpose and Type', 
                                   'Applicant Race/Eth', 
                                   'Applicant Income',
                                   'Applicant Sex'])
pieces = [totalapp5.ix[:, ["count","%count","amount","%amount",]]]
totalapp5 = pd.concat(pieces, axis=1, keys=['File closed for incompleteness'])
totalapp5

/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:11: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead
/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:38: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead
/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:39: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead


File closed for incompleteness  \
                                                                        count   
Loan Purpose and Type Home Improvement                                     89   
                      Purchase - Conventional                             681   
                      Purchase - Goverment                                267   
                      Refinancing                                        6397   
Applicant Race/Eth    aian                                                 31   
                      as                                                  703   
                      b                                                   143   
                      h                                                   809   
                      pi                                                   66   
                      w                                                  3890   
Applicant Income      Income Not Available                                674   
                      Low                                                 953   
                      Middle                                             1742   
                      Moderate                                           1475   
                      Upper                                              2590   
Applicant Sex         1                                                  4677   
                      2                                                  1513   
                      3                                                  1227   
                      4                                                    17   

                                                                         
                                               %count   amount  %amount  
Loan Purpose and Type Home Improvement            1.2    30405      1.2  
                      Purchase - Conventional     9.2   283546     10.8  
                      Purchase - Goverment        3.6    88870      3.4  
                      Refinancing                86.1  2229499     84.7  
Applicant Race/Eth    aian                        0.4     8557      0.3  
                      as                          9.5   235485      8.9  
                      b                           1.9    42075      1.6  
                      h                          10.9   224214      8.5  
                      pi                          0.9    19384      0.7  
                      w                          52.3  1457810     55.4  
Applicant Income      Income Not Available        9.1   238041      9.0  
                      Low                        12.8   181199      6.9  
                      Middle                     23.4   540178     20.5  
                      Moderate                   19.8   370028     14.1  
                      Upper                      34.8  1302874     49.5  
Applicant Sex         1                          62.9  1740723     66.1  
                      2                          20.4   447396     17.0  
                      3                          16.5   422311     16.0  
                      4                           0.2    21890      0.8

In [127]:
#######
# Action Type = 6
#######

df6 = df[df.action_type==6]
totalcount = df6.loan_amount.count()
totalamount = df6.loan_amount.sum()

### By Loan Purpose and Type

df6['loan_purtype'] = '_'
df6['loan_purtype'][(df6['loan_purpose'] == 1) & (df6['loan_type'] == 1)] = 'Purchase - Conventional'
df6['loan_purtype'][(df6['loan_purpose'] == 1) & (df6['loan_type'] > 1)] = 'Purchase - Goverment'
df6['loan_purtype'][df6['loan_purpose'] == 2] = 'Home Improvement'
df6['loan_purtype'][df6['loan_purpose'] == 3] = 'Refinancing'

loanpt = df6.groupby(['loan_purtype'])
loanpt = loanpt['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})
loanpt['%count'] = np.round(100*loanpt['count']/totalcount,1)
loanpt['%amount'] = np.round(100*loanpt['amount']/totalamount,1)
loanpt=loanpt[['count','%count','amount','%amount']]

### By Applicant Race/Ethnicity

race_eth  = df6.groupby(['race_eth'])
race_eth  = race_eth ['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

race_eth ['%count'] = np.round(100*race_eth['count']/totalcount,1)
race_eth ['%amount'] = np.round(100*race_eth['amount']/totalamount,1)
race_eth = race_eth [['count','%count','amount','%amount']]

###By Applicant Income

df6['applicant_income_percent_median'] = 100*df6.applicant_income/df6.applicant_income.median()
df6['applicant_income_cat'] = 'Income Not Available'
df6['applicant_income_cat'][df6['applicant_income_percent_median'] < 50] = 'Low'
df6['applicant_income_cat'][(df6['applicant_income_percent_median'] >= 50) & (df6['applicant_income_percent_median'] < 80)] = 'Moderate'
df6['applicant_income_cat'][(df6['applicant_income_percent_median'] >= 80) & (df6['applicant_income_percent_median'] < 120)] = 'Middle'
df6['applicant_income_cat'][df6['applicant_income_percent_median'] >= 120] = 'Upper'

income  = df6.groupby(['applicant_income_cat'])
income   = income['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

income ['%count'] = np.round(100*income['count']/totalcount,1)
income ['%amount'] = np.round(100*income['amount']/totalamount,1)
income = income [['count','%count','amount','%amount']]

###By Applicant Sex

sex  = df6.groupby(['applicant_sex'])
sex   = sex['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

sex ['%count'] = np.round(100*sex['count']/totalcount,1)
sex ['%amount'] = np.round(100*sex['amount']/totalamount,1)
sex = sex [['count','%count','amount','%amount']]

### Smashing Tables Together
frames = [loanpt, race_eth,income,sex]
totalapp6 = pd.concat(frames, keys=['Loan Purpose and Type', 
                                   'Applicant Race/Eth', 
                                   'Applicant Income',
                                   'Applicant Sex'])
pieces = [totalapp6.ix[:, ["count","%count","amount","%amount",]]]
totalapp6 = pd.concat(pieces, axis=1, keys=['Loan purchased by institution'])
totalapp6

/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:11: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead
/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:38: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead
/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:39: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead


Loan purchased by institution  \
                                                                       count   
Loan Purpose and Type Home Improvement                                   576   
                      Purchase - Conventional                           5905   
                      Purchase - Goverment                              7099   
                      Refinancing                                      26965   
Applicant Race/Eth    aian                                                81   
                      as                                                2686   
                      b                                                  436   
                      h                                                 3055   
                      pi                                                 248   
                      w                                                17155   
Applicant Income      Income Not Available                             10824   
                      Low                                               2811   
                      Middle                                            8785   
                      Moderate                                          7314   
                      Upper                                            10811   
Applicant Sex         1                                                19794   
                      2                                                 5718   
                      3                                                  824   
                      4                                                14209   

                                                                         
                                               %count   amount  %amount  
Loan Purpose and Type Home Improvement            1.4   156643      1.2  
                      Purchase - Conventional    14.6  1870706     14.3  
                      Purchase - Goverment       17.5  2402857     18.4  
                      Refinancing                66.5  8606369     66.0  
Applicant Race/Eth    aian                        0.2    26484      0.2  
                      as                          6.6   898491      6.9  
                      b                           1.1   135844      1.0  
                      h                           7.5   859720      6.6  
                      pi                          0.6    81154      0.6  
                      w                          42.3  5803813     44.5  
Applicant Income      Income Not Available       26.7  3561894     27.3  
                      Low                         6.9   516846      4.0  
                      Middle                     21.7  2822625     21.7  
                      Moderate                   18.0  1888182     14.5  
                      Upper                      26.7  4247028     32.6  
Applicant Sex         1                          48.8  6727297     51.6  
                      2                          14.1  1688278     13.0  
                      3                           2.0   285836      2.2  
                      4                          35.0  4335164     33.3

In [129]:
#Merging ALL
all_lenders = pd.concat([totalapp,totalapp1,totalapp2,totalapp3,totalapp4,totalapp5,totalapp6], axis=1)
all_lenders

Total Applications (1)          \
                                                                count  %count   
Loan Purpose and Type Home Improvement                           4781     2.0   
                      Purchase - Conventional                   32857    13.5   
                      Purchase - Goverment                      22240     9.2   
                      Refinancing                              182905    75.3   
Applicant Race/Eth    aian                                        767     0.3   
                      as                                        23920     9.9   
                      b                                          4053     1.7   
                      h                                         24012     9.9   
                      pi                                         1908     0.8   
                      w                                        142000    58.5   
Applicant Income      Income Not Available                      28473    11.7   
                      Low                                       26217    10.8   
                      Middle                                    57469    23.7   
                      Moderate                                  49857    20.5   
                      Upper                                     80767    33.3   
Applicant Sex         1                                        159485    65.7   
                      2                                         50394    20.8   
                      3                                         17527     7.2   
                      4                                         15377     6.3   

                                                                  \
                                                 amount  %amount   
Loan Purpose and Type Home Improvement          1038687      1.3   
                      Purchase - Conventional  12419787     15.1   
                      Purchase - Goverment      7551607      9.2   
                      Refinancing              61055963     74.4   
Applicant Race/Eth    aian                       231319      0.3   
                      as                        7808190      9.5   
                      b                         1139822      1.4   
                      h                         6385159      7.8   
                      pi                         570463      0.7   
                      w                        48384565     59.0   
Applicant Income      Income Not Available     11876933     14.5   
                      Low                       4777981      5.8   
                      Middle                   17701587     21.6   
                      Moderate                 12375557     15.1   
                      Upper                    35333986     43.1   
Applicant Sex         1                        54930806     66.9   
                      2                        14172171     17.3   
                      3                         5939329      7.2   
                      4                         7023738      8.6   

                                               Loan originated          \
                                                         count  %count   
Loan Purpose and Type Home Improvement                    2315     1.7   
                      Purchase - Conventional            18452    13.7   
                      Purchase - Goverment               10545     7.8   
                      Refinancing                       103541    76.8   
Applicant Race/Eth    aian                                 417     0.3   
                      as                                 14274    10.6   
                      b                                   2219     1.6   
                      h                                  12799     9.5   
                      pi                                  1027     0.8   
                      w                                  86821    64.4   
Applicant Income      Income

In [130]:
all_lenders.to_csv('allLenders.csv')